In [5]:
""" get some test images """
import os

DIR = '/Users/garyburgmann/mot_challenge/MOT17Det/train/MOT17-05/img1'

images = sorted(os.listdir(DIR))

print(images[:10])

['000001.jpg', '000002.jpg', '000003.jpg', '000004.jpg', '000005.jpg', '000006.jpg', '000007.jpg', '000008.jpg', '000009.jpg', '000010.jpg']


In [14]:
"""
use tensorflow hub models 

Model name	Speed (ms)	COCO mAP	Outputs
CenterNet HourGlass104 512x512	70	41.9	Boxes
CenterNet HourGlass104 Keypoints 512x512	76	40.0/61.4	Boxes/Keypoints
CenterNet HourGlass104 1024x1024	197	44.5	Boxes
CenterNet HourGlass104 Keypoints 1024x1024	211	42.8/64.5	Boxes/Keypoints
CenterNet Resnet50 V1 FPN 512x512	27	31.2	Boxes
CenterNet Resnet50 V1 FPN Keypoints 512x512	30	29.3/50.7	Boxes/Keypoints
CenterNet Resnet101 V1 FPN 512x512	34	34.2	Boxes
CenterNet Resnet50 V2 512x512	27	29.5	Boxes
CenterNet Resnet50 V2 Keypoints 512x512	30	27.6/48.2	Boxes/Keypoints
EfficientDet D0 512x512	39	33.6	Boxes
EfficientDet D1 640x640	54	38.4	Boxes
EfficientDet D2 768x768	67	41.8	Boxes
EfficientDet D3 896x896	95	45.4	Boxes
EfficientDet D4 1024x1024	133	48.5	Boxes
EfficientDet D5 1280x1280	222	49.7	Boxes
EfficientDet D6 1280x1280	268	50.5	Boxes
EfficientDet D7 1536x1536	325	51.2	Boxes
SSD MobileNet v2 320x320	19	20.2	Boxes
SSD MobileNet V1 FPN 640x640	48	29.1	Boxes
SSD MobileNet V2 FPNLite 320x320	22	22.2	Boxes
SSD MobileNet V2 FPNLite 640x640	39	28.2	Boxes
SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	46	34.3	Boxes
SSD ResNet50 V1 FPN 1024x1024 (RetinaNet50)	87	38.3	Boxes
SSD ResNet101 V1 FPN 640x640 (RetinaNet101)	57	35.6	Boxes
SSD ResNet101 V1 FPN 1024x1024 (RetinaNet101)	104	39.5	Boxes
SSD ResNet152 V1 FPN 640x640 (RetinaNet152)	80	35.4	Boxes
SSD ResNet152 V1 FPN 1024x1024 (RetinaNet152)	111	39.6	Boxes
Faster R-CNN ResNet50 V1 640x640	53	29.3	Boxes
Faster R-CNN ResNet50 V1 1024x1024	65	31.0	Boxes
Faster R-CNN ResNet50 V1 800x1333	65	31.6	Boxes
Faster R-CNN ResNet101 V1 640x640	55	31.8	Boxes
Faster R-CNN ResNet101 V1 1024x1024	72	37.1	Boxes
Faster R-CNN ResNet101 V1 800x1333	77	36.6	Boxes
Faster R-CNN ResNet152 V1 640x640	64	32.4	Boxes
Faster R-CNN ResNet152 V1 1024x1024	85	37.6	Boxes
Faster R-CNN ResNet152 V1 800x1333	101	37.4	Boxes
Faster R-CNN Inception ResNet V2 640x640	206	37.7	Boxes
Faster R-CNN Inception ResNet V2 1024x1024	236	38.7	Boxes
Mask R-CNN Inception ResNet V2 1024x1024	301	39.0/34.6	Boxes/Masks
"""
import tensorflow_hub as hub

module_handle = (
    "https://tfhub.dev/tensorflow/efficientdet/d3/1"
    # "https://tfhub.dev/tensorflow/centernet/resnet50v1_fpn_512x512/1",
    # "https://tfhub.dev/tensorflow/faster_rcnn/resnet101_v1_1024x1024/1"
    # "https://tfhub.dev/tensorflow/efficientdet/d2/1"
    # "https://tfhub.dev/tensorflow/centernet/hourglass_512x512/1"
    # "https://tfhub.dev/tensorflow/efficientdet/d7/1"
    # "https://tfhub.dev/tensorflow/retinanet/resnet101_v1_fpn_640x640/1"
    # "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"
) #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]

clf = hub.load(module_handle)
print('done')

_conditional_losses_149230) with ops with custom gradients. Will likely fail if a gradient is requested.
done


In [16]:
import time
import io
from typing import Dict

import cv2
import numpy as np
import tensorflow as tf


def format_results(detections: Dict, num_detections: int):
    # Get all output details
    boxes = detections['detection_boxes']
    classes = detections['detection_classes']
    scores = detections['detection_scores']

    results = [
        {
            'bounding_box': boxes[i],
            'class_id': int(classes[i]),
            # 'class': self._labels[classes[i]].lower(),
            'score': scores[i]
        }
        for i in range(num_detections)
        if int(classes[i]) == 0
        # and scores[i] >= 0.51
    ]
    return results

for x in images[:5]:
    img = cv2.imread(f'{DIR}/{x}')
    converted_img  = tf.image.convert_image_dtype(img, tf.uint8)[tf.newaxis, ...]
    try:
        start = time.time()
        results = clf(converted_img)
        print('inference time: ', time.time() - start)
        # print('results: ', results)

        num_detections = int(results.pop('num_detections'))
        print('num_detections: ', num_detections)

        detections = {
            key: value[0, :num_detections].numpy()
            for key, value in results.items()
        }

        results = format_results(detections, num_detections)
        # print('\n\nresults: ', results)
    except Exception as exc:
        print('exc: ', exc)

#     cv2.imshow('frame', img)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cv2.destroyAllWindows()

inference time:  2.8699750900268555
num_detections:  100
inference time:  2.9946470260620117
num_detections:  100
inference time:  3.1445727348327637
num_detections:  100
inference time:  3.310837984085083
num_detections:  100
inference time:  3.3334498405456543
num_detections:  100
